# Проект Дашборд конверсий

### подключаем библиотеки

In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
from dotenv import load_dotenv
import os

### получаем данные

In [ ]:
load_dotenv()

# Получаем значения переменных окружения
API_URL = os.getenv('API_URL')
DATE_BEGIN = os.getenv('DATE_BEGIN')
DATE_END = os.getenv('DATE_END')

# Формируем URL для запросов
url_visits = f'{API_URL}/visits?begin={DATE_BEGIN}&end={DATE_END}'
url_registrations = f'{API_URL}/registrations?begin={DATE_BEGIN}&end={DATE_END}'

# Выполняем запросы
response_visits = requests.get(url_visits)
data_visits = response_visits.json()

response_registrations = requests.get(url_registrations)
data_registrations = response_registrations.json()

### изучаем данные

In [ ]:
df_visits = pd.DataFrame(data_visits)
df_visits.head(10)

In [ ]:
df_visits.tail(10)

In [ ]:
df_registrations = pd.DataFrame(data_registrations)
df_registrations.head(10)

In [ ]:
df_registrations.tail(10)

### краткая сводка по данным

In [ ]:
df_visits.describe()

In [ ]:
df_registrations.describe()

### Расчет метрик

In [ ]:
# Преобразуем datetime в формат даты (без времени)
df_visits['date'] = pd.to_datetime(df_visits['datetime']).dt.date
df_registrations['date'] = pd.to_datetime(df_registrations['datetime']).dt.date

# Убираем визиты, которые были сделаны ботами
df_visits = df_visits[~df_visits['user_agent'].str.contains('bot')]

# Сгруппируем данные по дате и платформам для визитов
df_visits = df_visits.sort_values(by=['visit_id', 'date']).drop_duplicates(subset=['visit_id'], keep='last')
visits_grouped = df_visits.groupby(['date', 'platform']).size().reset_index(name='visits')

# Сгруппируем данные регистраций по дате и платформам
registrations_grouped = (df_registrations
                         .groupby(['date', 'platform'])
                         .agg(registrations=('user_id', 'count'))
                         .reset_index())

# Объединяем датафреймы визитов и регистраций
merged_df = pd.merge(visits_grouped, registrations_grouped, on=['date', 'platform'], how='outer').fillna(0)

# Рассчитаем конверсию
merged_df['conversion'] = (merged_df['registrations'] / merged_df['visits'].replace(0, np.nan)) * 100

# Приводим датафрейм в нужный формат
merged_df['date_group'] = merged_df['date'].astype('datetime64[ms]')
final_df = merged_df[['date_group', 'platform', 'visits', 'registrations', 'conversion']]

# Сортируем данные по дате
final_df = final_df.sort_values(by='date_group')

# Сохраняем датафрейм в формате JSON
final_df.to_json('./conversion.json')

In [ ]:
final_df.head()

In [ ]:
final_df.tail(20)

### добавляем рекламу

In [ ]:
# Загрузка данных о рекламных кампаниях
ads_df = pd.read_csv('ads.csv')

In [ ]:
ads_df['date'] = pd.to_datetime(ads_df['date']).dt.date

# Сагрегируем данные по дате и utm_campaign, суммируем затраты
ads_grouped = (ads_df
               .groupby(['date', 'utm_campaign'])
               .agg(cost=('cost', 'sum'))
               .reset_index())

# Агрегируем по дате
ads_grouped = (ads_grouped
               .groupby('date')
               .agg(cost=('cost', 'sum'),
                    utm_campaign=('utm_campaign', lambda x: ', '.join(x) if not x.empty else 'none'))
               .reset_index())

# Переименуем 'date' в 'date_group'
ads_grouped.rename(columns={'date': 'date_group'}, inplace=True)

# Объединяем датафрейм с конверсиями
final_df['date_group'] = pd.to_datetime(final_df['date_group']).dt.date
merged_final = pd.merge(final_df.groupby('date_group').agg(
                          visits=('visits', 'sum'),
                          registrations=('registrations', 'sum')).reset_index(), 
                        ads_grouped, 
                        on='date_group', 
                        how='left')

# Заполняем отсутствующие значения
merged_final['utm_campaign'] = merged_final['utm_campaign'].fillna('none')
merged_final['cost'] = merged_final['cost'].fillna(0)

# Оставляем нужные колонки в итоговом датафрейме
final_result = merged_final[['date_group', 'visits', 'registrations', 'cost', 'utm_campaign']]

# Сортируем данные по date_group
final_result = final_result.sort_values(by='date_group')

# Сохраняем итоговый датафрейм в формате JSON
final_result.to_json('./ads.json')

In [ ]:
final_result.head(10)

In [ ]:
final_result.tail(20)

## Визуализация

### Итоговые визиты

In [ ]:
plt.figure(figsize=(22, 10))
bar_plot = sns.barplot(data=final_result, x='date_group', y='visits', color='skyblue')
plt.title('Total visits')
plt.xlabel('date_group')
plt.ylabel('visits')
final_result['date_group'] = pd.to_datetime(final_result['date_group'])
monthly_ticks = final_result[final_result['date_group'].dt.is_month_start]
monthly_ticks_positions = monthly_ticks.index
plt.xticks(monthly_ticks_positions, labels=monthly_ticks['date_group'].dt.strftime('%Y-%m'), rotation=0, ha='left')
for p in bar_plot.patches:
    bar_plot.annotate(f'{int(p.get_height())}',
                        (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='baseline', color='black', fontsize=6, xytext=(0, 5),
                        textcoords='offset points')
    
plt.tight_layout()
plt.savefig(f'./charts/total_visits')
plt.show()

### Итоговые визиты с разбивкой по платформам

In [ ]:
final_df['date_group'] = pd.to_datetime(final_df['date_group'])

# Сгруппируем данные по дате и платформам
stacked_data = final_df.pivot_table(index='date_group', columns='platform', values='visits', fill_value=0)

# Построим stacked bar chart
plt.figure(figsize=(22, 10))
stacked_data.plot(kind='bar', stacked=True, ax=plt.gca())

# Настраиваем заголовки оси
plt.title('Visits by Platform (Stacked)')
plt.xlabel('date_group')
plt.ylabel('visits')

monthly_ticks = stacked_data.index[stacked_data.index.day == 1]  # Только первые дни месяца
monthly_tick_positions = [stacked_data.index.get_loc(date) for date in monthly_ticks]
plt.xticks(monthly_tick_positions, labels=monthly_ticks.strftime('%Y-%m'), rotation=0, ha='left')

# Добавляем легенду
plt.legend(title='platform', bbox_to_anchor=(0.94, 1), loc='upper left')

plt.tight_layout()
plt.savefig('./charts/total_visits_by_platform.png')
plt.show()

### Итоговые регистрации

In [ ]:
plt.figure(figsize=(22, 10))
bar_plot = sns.barplot(data=final_result, x='date_group', y='registrations', color='skyblue')
plt.title('Total registrations')
plt.xlabel('date_group')
plt.ylabel('registrations')
final_result['date_group'] = pd.to_datetime(final_result['date_group'])
monthly_ticks = final_result[final_result['date_group'].dt.is_month_start]
monthly_ticks_positions = monthly_ticks.index
plt.xticks(monthly_ticks_positions, labels=monthly_ticks['date_group'].dt.strftime('%Y-%m'), rotation=0, ha='left')
for p in bar_plot.patches:
    bar_plot.annotate(f'{int(p.get_height())}',
                        (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='baseline', color='black', fontsize=6, xytext=(0, 5),
                        textcoords='offset points')
    
plt.tight_layout()
plt.savefig(f'./charts/total_registrations')
plt.show()

### Итоговые регистрации с разбивкой по платформе

In [ ]:
final_df['date_group'] = pd.to_datetime(final_df['date_group'])

# Сгруппируем данные по дате и платформам
stacked_data = final_df.pivot_table(index='date_group', columns='platform', values='registrations', fill_value=0)

# Построим stacked bar chart
plt.figure(figsize=(22, 10))
stacked_data.plot(kind='bar', stacked=True, ax=plt.gca())

# Настраиваем заголовки оси
plt.title('Registrations by Platform (Stacked)')
plt.xlabel('date_group')
plt.ylabel('visits')

monthly_ticks = stacked_data.index[stacked_data.index.day == 1]  # Только первые дни месяца
monthly_tick_positions = [stacked_data.index.get_loc(date) for date in monthly_ticks]
plt.xticks(monthly_tick_positions, labels=monthly_ticks.strftime('%Y-%m'), rotation=0, ha='left')

# Добавляем легенду
plt.legend(title='platform', bbox_to_anchor=(0.94, 1), loc='upper left')

plt.tight_layout()
plt.savefig('./charts/total_registrations_by_platform.png')
plt.show()

### Общая конверсия

In [ ]:
final_df['date_group'] = pd.to_datetime(final_df['date_group'])

conversion_data = final_df.groupby('date_group')['conversion'].mean().reset_index()

# Настраиваем график
plt.figure(figsize=(22, 10))
plt.plot(conversion_data['date_group'], conversion_data['conversion'], marker='o', linestyle='dashed')

# Добавляем подписанные значения на график
for idx, row in conversion_data.iterrows():
    plt.text(row['date_group'], row['conversion'], f"{row['conversion']:.2f}%", 
             ha='center', va='bottom', fontsize=9)

# Настроим заголовки и метки
plt.title('Overall Conversion')
plt.xlabel('Date')
plt.ylabel('Conversion (%)')
plt.xticks(fontsize=14)
plt.grid()

# Сохраняем и показываем график
plt.tight_layout()
plt.savefig('./charts/overall_conversiont.png')
plt.show()

### конверсии для каждой платформы

In [ ]:
final_df['date_group'] = pd.to_datetime(final_df['date_group'])

# Создаем сводную таблицу для получения итоговых конверсий по дням для каждой платформы
conversion_data = final_df.groupby(['date_group', 'platform'])['conversion'].mean().reset_index()

platforms = conversion_data['platform'].unique()

# Настраиваем графики
fig, axs = plt.subplots(len(platforms), 1, figsize=(12, 18))

for i, platform in enumerate(platforms):
    platform_data = conversion_data[conversion_data['platform'] == platform]
    axs[i].plot(platform_data['date_group'], platform_data['conversion'], marker='o', color='skyblue', label='Conversion')

    # Добавляем подписанные значения на график
    for idx, row in platform_data.iterrows():
        axs[i].text(row['date_group'], row['conversion'], f"{row['conversion']:.2f}%", 
                    ha='center', va='bottom', fontsize=8)

    # Настроим заголовки и метки для каждого графика
    axs[i].set_title(f'Conversion for {platform}')
    axs[i].set_ylabel('Conversion (%)')
    axs[i].grid()
    axs[i].legend(title='Conversion')

# Настройка оси X
plt.xlabel('Date')

plt.tight_layout()
plt.savefig('./charts/conversion_by_platforms.png')
plt.show()

### стоимости рекламы

In [ ]:
final_result['date_group'] = pd.to_datetime(final_result['date_group'])

# Настройка графика
plt.figure(figsize=(16, 8))
plt.plot(final_result['date_group'], final_result['cost'], marker='o', linestyle='-', color='skyblue')

for idx, row in final_result.iterrows():
    if row['cost'] == 0:
            continue
    plt.text(row['date_group'], row['cost'], f"{row['cost']:.0f}RUB", 
             ha='center', va='bottom', fontsize=8)

# Добавление подписей и заголовка
plt.title('Aggregated Ad Campaign Cost')
plt.xlabel('Date')
plt.ylabel('Cost(RUB)')
plt.xticks(rotation=45)  # Поворот меток оси X для удобства чтения
plt.grid()
plt.tight_layout()

# Сохранение и отображение графика
plt.savefig('./charts/ad_cost.png')
plt.show()

### Визиты за весь период с цветовым выделением рекламной кампании

In [ ]:
visits_color = final_result.copy()
visits_color['date_group'] = pd.to_datetime(visits_color['date_group'])

# Создаем колонку с номерами кампаний
visits_color['campaigns_n'] = (visits_color['utm_campaign'] != visits_color['utm_campaign'].shift()).cumsum()

# Фильтруем данные, исключая строки с 'none'
campaign_periods = (visits_color
                    .dropna(subset=['utm_campaign'])
                    .loc[visits_color['utm_campaign'] != 'none']  # Исключаем 'none'
                    .groupby(['campaigns_n','utm_campaign'])
                    .agg(start=('date_group', 'min'), finish=('date_group', 'max'))
                    .reset_index()
                   )

# Строим график
visits_color.plot(x='date_group', y='visits', kind='line', figsize=(16, 8), 
                  linewidth=0.8, color='black', label='visits', marker='o', markersize=5)
plt.axhline(visits_color['visits'].mean(), linestyle='--', color='navy', label='avg_visits')

# Добавляем цветные области для каждой рекламной кампании
colors = ['salmon', 'peachpuff', 'palegreen', 'powderblue', 'thistle']
for i, row in campaign_periods.iterrows():
    plt.axvspan(row['start'], row['finish'], color=colors[i % len(colors)], alpha=0.3, label=row['utm_campaign'])

# Оформление графика
plt.grid(axis='y', linestyle=':', color='grey')
plt.legend(loc='best')
plt.title('Visits during marketing active days')
plt.xlabel('date_group')
plt.ylabel('visits')
plt.tight_layout()
plt.savefig('./charts/visits_during_campaigns.png')
plt.show()

### Регистрации за весь период с цветовым выделением рекламной кампании

In [ ]:
registrations_color = final_result.copy()
registrations_color['date_group'] = pd.to_datetime(registrations_color['date_group'])

# Создаем колонку с номерами кампаний
registrations_color['campaigns_n'] = (registrations_color['utm_campaign'] != registrations_color['utm_campaign'].shift()).cumsum()

# Фильтруем данные, исключая строки с 'none'
campaign_periods = (registrations_color
                    .dropna(subset=['utm_campaign'])
                    .loc[registrations_color['utm_campaign'] != 'none']  # Исключаем 'none'
                    .groupby(['campaigns_n','utm_campaign'])
                    .agg(start=('date_group', 'min'), finish=('date_group', 'max'))
                    .reset_index()
                   )

# Строим график
registrations_color.plot(x='date_group', y='registrations', kind='line', figsize=(16, 8), 
                  linewidth=0.8, color='black', label='registrations', marker='o', markersize=5)
plt.axhline(registrations_color['registrations'].mean(), linestyle='--', color='navy', label='avg_registrations')

# Добавляем цветные области для каждой рекламной кампании
colors = ['salmon', 'peachpuff', 'palegreen', 'powderblue', 'thistle']
for i, row in campaign_periods.iterrows():
    plt.axvspan(row['start'], row['finish'], color=colors[i % len(colors)], alpha=0.3, label=row['utm_campaign'])

# Оформление графика
plt.grid(axis='y', linestyle=':', color='grey')
plt.legend(loc='best')
plt.title('Registrations during marketing active days')
plt.xlabel('date_group')
plt.ylabel('registrations')
plt.tight_layout()
plt.savefig('./charts/registrations_during_campaigns.png')
plt.show()

### Зависимость посещений от рекламы

In [ ]:
final_result['date_group'] = pd.to_datetime(final_result['date_group'])
fig, ax1 = plt.subplots(figsize=(22, 10))

ax1.bar(final_result['date_group'], final_result['visits'], label='Total Visits', color='skyblue')
ax1.set_xlabel('date_group')
ax1.set_ylabel('visits')
ax1.tick_params(axis='y')
ax1.set_title('Total Visits and Ad Campaign Cost')
ax1.grid(True)

ax2 = ax1.twinx()
ax2.plot(final_result['date_group'], final_result['cost'], color='red', label='Ad Campaign Cost')
ax2.set_ylabel('Cost (RUB)')
ax2.tick_params(axis='y')
plt.savefig('./charts/total_visits_and_ad_campaign_cost.png')
plt.show()

### Зависимость регистраций от рекламы

In [ ]:
final_result['date_group'] = pd.to_datetime(final_result['date_group'])
fig, ax1 = plt.subplots(figsize=(22, 10))

ax1.bar(final_result['date_group'], final_result['registrations'], label='Total Registrations', color='palegreen')
ax1.set_xlabel('date_group')
ax1.set_ylabel('registrations')
ax1.tick_params(axis='y')
ax1.set_title('Total Registrations and Ad Campaign Cost')
ax1.grid(True)

ax2 = ax1.twinx()
ax2.plot(final_result['date_group'], final_result['cost'], color='red', label='Ad Campaign Cost')
ax2.set_ylabel('Cost (RUB)')
ax2.tick_params(axis='y')
plt.savefig('./charts/total_registrations_and_ad_campaign_cost.png')
plt.show()